<a href="https://colab.research.google.com/github/Sheha1218/Sinhala-call-center-assistant-for-Sri-Lanka-telecommunication-industry/blob/main/research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers peft trl -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
import transformers
from huggingface_hub import login
from transformers import AutoModelForCausalLM, BitsAndBytesConfig,AutoTokenizer, AutoModelForCausalLM,GemmaTokenizer
from collections import Counter, defaultdict
from peft import LoraConfig,PeftConfig,PeftModel
from trl import SFTTrainer
import pandas as pd
from google.colab import userdata
from huggingface_hub import login

In [ ]:
hf_token = userdata.get('hf_token')
os.environ['HF_TOKEN'] = hf_token
login(token=hf_token)
os.environ['WANDB_DISABLED'] = 'True'

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
data=pd.read_csv('/content/data.csv')

In [ ]:
model_name='google/gemma-2b'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             device_map={"":0}
                                             )

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules =['q_proj','o_proj','k_proj','v_proj',
                    'gate_proj','up_proj','down_proj'],
    task_type="CAUSAL_LM"
)

In [ ]:
data=pd.read_csv('/content/data.csv')

In [ ]:
data.sample()

,Trasaction.,process.
75,Ownership transfer.,ownsership transfer එක කරගන්න ඔනේ කෙනේ අයිති ක...


In [ ]:
from datasets import Dataset, load_dataset
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
data=Dataset.from_pandas(data)

In [ ]:
print(type(data))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
print(type(data[0]["process."]))
print(data[0]["process."])


<class 'str'>
Sim change එකක් කරගනිද්දි original customer brunch එකට visit කරන්න ඔනේ. අනිවර්යෙන්ම original customer NIC හො Driving license එක අරගෙන brunch එකට යන්න ඕනේ. අපි හිතමු NIC එක කියලා unclear හරි dammage අපිට NIC එක verified කරන්න බෑ .ඒ විතරක් නෙමෙයි NIC එක නැත්නම් අපි Driving license  එක ගෙනියන්ව කියමුකො එකගෙනිච්චත්  same එත් එක clear තියෙන්න ඔනේ. එක expired වෙලා තියෙන්න බෑ incase expire වෙලානම් අපි එක accept  කරන්නෙත් නැ. ඔය දෙකම ලග නැත්නම් අපිට එයාට option එකක් දෙන්න පුලුවන් ඒ තමය් Passport එක. ඒත් එකත් expired වෙලා තියෙන්න බෑ. තව perority customer කෙනෙක් නෙමෙයි නම් රුපියල් 100ක් change වෙනවා ඒ වගේම තමන්ගි නමට SIM එක තියෙන්නත් ඔනේ සර් . Sim එක active වෙන්න පැය 3ක් වගේ යනවා.


In [ ]:
print(data.column_names)

['Trasaction.', 'process.']


In [ ]:
text = data["process."]

In [ ]:
bad_rows = [i for i in range(len(data)) if not data[i]["process."]]
print(bad_rows)

[19, 20, 39, 57, 58, 73]


In [ ]:
print(data[0]["process."])
print(type(data[0]["process."]))


Sim change එකක් කරගනිද්දි original customer brunch එකට visit කරන්න ඔනේ. අනිවර්යෙන්ම original customer NIC හො Driving license එක අරගෙන brunch එකට යන්න ඕනේ. අපි හිතමු NIC එක කියලා unclear හරි dammage අපිට NIC එක verified කරන්න බෑ .ඒ විතරක් නෙමෙයි NIC එක නැත්නම් අපි Driving license  එක ගෙනියන්ව කියමුකො එකගෙනිච්චත්  same එත් එක clear තියෙන්න ඔනේ. එක expired වෙලා තියෙන්න බෑ incase expire වෙලානම් අපි එක accept  කරන්නෙත් නැ. ඔය දෙකම ලග නැත්නම් අපිට එයාට option එකක් දෙන්න පුලුවන් ඒ තමය් Passport එක. ඒත් එකත් expired වෙලා තියෙන්න බෑ. තව perority customer කෙනෙක් නෙමෙයි නම් රුපියල් 100ක් change වෙනවා ඒ වගේම තමන්ගි නමට SIM එක තියෙන්නත් ඔනේ සර් . Sim එක active වෙන්න පැය 3ක් වගේ යනවා.
<class 'str'>


In [ ]:
data = data.filter(
    lambda x: x["process."] is not None and x["process."] != ""
)


Filter:   0%|          | 0/89 [00:00<?, ? examples/s]

In [ ]:
data = data.map(
    lambda x: tokenizer(
        text=x["process."],
        truncation=True,
        padding="max_length",
        max_length=512,
    ),
    batched=False
)


Map:   0%|          | 0/83 [00:00<?, ? examples/s]

In [ ]:
data['process.']

Column(['Sim change එකක් කරගනිද්දි original customer brunch එකට visit කරන්න ඔනේ. අනිවර්යෙන්ම original customer NIC හො Driving license එක අරගෙන brunch එකට යන්න ඕනේ. අපි හිතමු NIC එක කියලා unclear හරි dammage අපිට NIC එක verified කරන්න බෑ .ඒ විතරක් නෙමෙයි NIC එක නැත්නම් අපි Driving license  එක ගෙනියන්ව කියමුකො එකගෙනිච්චත්  same එත් එක clear තියෙන්න ඔනේ. එක expired වෙලා තියෙන්න බෑ incase expire වෙලානම් අපි එක accept  කරන්නෙත් නැ. ඔය දෙකම ලග නැත්නම් අපිට එයාට option එකක් දෙන්න පුලුවන් ඒ තමය් Passport එක. ඒත් එකත් expired වෙලා තියෙන්න බෑ. තව perority customer කෙනෙක් නෙමෙයි නම් රුපියල් 100ක් change වෙනවා ඒ වගේම තමන්ගි නමට SIM එක තියෙන්නත් ඔනේ සර් . Sim එක active වෙන්න පැය 3ක් වගේ යනවා.', 'Original customer branch එකට visit කරන්න ඔනේ එයගේ NIC, Driving license or Passport  එකත් එක්ක ඒ වගේම රුපියක් 100 charge වෙනවා. පැය 3 වගේ කාලයක් යනවා. ඊට පස්සේ තමයි ඔබතුමාගෙ SIM  එක phone  එකට දාගන්න ඕනේ.', 'Sim change එක කරන්න original customer brunch එකට එන්න ඔනේ brunch එකට එනකොට NIC එක හරි Driving license

In [ ]:
def formatting_func(example):
  text=f"process.: {example['process.'][0]}\Trasaction:{example['Trasaction.'][0]}"
  return[text]

<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipython-input-551813273.py:2: SyntaxWarning: invalid escape sequence '\T'
  text=f"process.: {example['process.'][0]}\Trasaction:{example['Trasaction.'][0]}"


In [ ]:
from numpy import gradient
trainer =SFTTrainer(model=model,
                    train_dataset=data,
                    args=transformers.TrainingArguments(
                        per_device_train_batch_size =1,
                        gradient_accumulation_steps=4,
                        warmup_steps=2,
                        max_steps=150,
                        learning_rate=2e-4,
                        bf16=True,
                        logging_steps=1,
                        output_dir='output',
                        optim='paged_adamw_8bit',
                        report_to='none'

                    ),
                    peft_config=lora_config,
                    formatting_func=formatting_func,
                    )

Truncating train dataset:   0%|          | 0/83 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
1,12.252000
2,13.822800
3,11.662800
4,11.300000
5,10.843600
6,9.879200
7,9.983100
8,8.493500
9,8.881500
10,8.849800


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=150, training_loss=4.177030258178711, metrics={'train_runtime': 1668.1722, 'train_samples_per_second': 0.36, 'train_steps_per_second': 0.09, 'total_flos': 3628254143447040.0, 'train_loss': 4.177030258178711, 'epoch': 7.144578313253012})

In [ ]:
output_dir = './gemma-2-9b-finetuned'
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./gemma-2-9b-finetuned/tokenizer_config.json',
 './gemma-2-9b-finetuned/special_tokens_map.json',
 './gemma-2-9b-finetuned/tokenizer.model',
 './gemma-2-9b-finetuned/added_tokens.json',
 './gemma-2-9b-finetuned/tokenizer.json')